In [44]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# sources
# https://gist.github.com/MichelleDalalJian/453c68e7fde2b2996c8b598c988c09d3
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
# https://stackoverflow.com/questions/12850345/how-do-i-combine-two-dataframes
# https://www.geeksforgeeks.org/create-a-dictionary-with-list-comprehension-in-python/
# https://stackoverflow.com/questions/38233838/selecting-second-child-using-beautifulsoup
# https://stackoverflow.com/questions/24108507/beautiful-soup-resultset-object-has-no-attribute-find-all
# https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class
# https://stackoverflow.com/questions/63177365/why-is-concat-adding-time-to-the-date-when-concat-two-dataframes
# https://stackoverflow.com/questions/12451997/beautifulsoup-gettext-from-between-p-not-picking-up-subsequent-paragraphs

In [43]:
MULT_COUNTY_JAIL_BOOKINGS_URL = "http://www.mcso.us/PAID/Home/SearchResults"
CSV_LOCATION = 'data/data.csv'

prev_data = pd.read_csv(CSV_LOCATION, index_col=False)

response = requests.post(MULT_COUNTY_JAIL_BOOKINGS_URL)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')
table_input = table.prettify()

table_to_df = pd.read_html(table_input, parse_dates=[1])[0]

table_to_df = table_to_df.sort_values(by=["Booking Date", "Name"], ascending=[False, True])
# newdf = pd.concat([prev_data.astype({'Booking Date': 'str'}), table_to_df.astype({'Booking Date': 'str'})])
# newdf = newdf.drop_duplicates().reset_index(drop=True).sort_values(by="Booking Date", ascending=False)
# newdf = newdf.drop_duplicates(keep=False).sort_values(by=["Booking Date", "Name"])

table_to_df.to_csv('data/updated.csv', index=False)

In [55]:
def generate_charges_info(inmate_link):
    inmate_response = requests.get(inmate_link)
    soup = BeautifulSoup(inmate_response.text, 'html.parser')

    charge_infos = soup.find(id='charge-info')
    cases = charge_infos.select('div > h3')

    charge_items = []
    charges = charge_infos.select('div > div > ol')
    for charge in charges:
        for el in charge.find_all('li'):
            charge_type = el.find('span').getText()
            charge_bail = el.find(class_='charge-bail-display').text
            charge_status = el.find(class_='charge-status-value').text
            charge_item = {
                'Type': charge_type,
                'Bail': charge_bail,
                'Status': charge_status
            }
            charge_items.append(charge_item)
    return charge_items

In [65]:
def generate_charge_type_totals(charge_items):
    charge_types = [charge['Type'] for charge in charge_items]
    return pd.Series(charge_types).value_counts().to_dict()

In [72]:
links = table.find_all('a')
INMATE_COLUMNS = ['SWIS ID', 'Name', 'Age', 'Gender', 'Race', 'Height', 'Weight', 'Hair', 'Eyes', 'Arresting Agency', 'Booking Date', 'Assigned Facility', 'Projected Release Date']
jailed_data = []
for link in tqdm(links[:3]):
    booking_url = link.get('href')
    inmate_link = "http://www.mcso.us" + booking_url
    
    # go to inmate's page
    inmate_response = requests.get(inmate_link)
    soup = BeautifulSoup(inmate_response.text, 'html.parser')
    table_input = soup.find('table').prettify()
    booking_table = pd.read_html(table_input)[0][1]
    
    inmate_data = dict(zip(INMATE_COLUMNS, booking_table))
    charge_items = generate_charges_info(inmate_link)
    inmate_data['Charge Type Counts'] = generate_charge_type_totals(charge_items)
    inmate_data['Charges'] = charge_items
    jailed_data.append(inmate_data)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.24it/s]


In [74]:
inmates_df = pd.DataFrame(jailed_data).sort_values(by=['Booking Date', 'Name'], ascending=[False, True])
inmates_df

,SWIS ID,Name,Age,Gender,Race,Height,Weight,Hair,Eyes,Arresting Agency,Booking Date,Assigned Facility,Projected Release Date,Charge Type Counts,Charges
0,820510,"Abdi, Adan Osman",26,Male,Black,5 ft 6 in,140 lbs,Black,Brown,"Portland Police, Central Precinct",07/20/2022 06:12 AM,MCDC,Unknown,"{'ATT ROBBERY II (C Felony)': 1, 'ASSAULT III ...","[{'Type': 'ATT ROBBERY II (C Felony)', 'Bail':..."
1,828509,"Acevedohernandez, Jairo",20,Male,White,5 ft 7 in,157 lbs,Black,Brown,"Portland Police, Other",07/06/2021 11:40 PM,MCDC,Unknown,"{'RECK ENDANGER (A Misdemeanor)': 9, 'TAMP W/W...","[{'Type': 'MURDER II (A Felony)', 'Bail': 'Bai..."
2,782353,"Adams, Tawnisha Nicole",26,Female,White,5 ft 7 in,180 lbs,Blonde,Blue,"Portland Police, Central Precinct",06/09/2022 08:01 AM,MCDC,Unknown,"{'PCS 1 FEL (B Felony)': 2, 'POSS METH FEL (C ...","[{'Type': 'DEL HEROIN (A Felony)', 'Bail': 'Ba..."


In [70]:
inmates_df.to_csv('data/inmates.csv')

In [38]:
df1=pd.DataFrame({'A':[1,2,3,3],'B':[2,3,4,4]})
df2=pd.DataFrame({'A':[1],'B':[2]})
# res = pd.concat([df1, df2]).drop_duplicates(keep=False)
res = df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

print(df1)
print(df2)

res

   A  B
0  1  2
1  2  3
2  3  4
3  3  4
   A  B
0  1  2


,A,B
1,2,3
2,3,4
3,3,4
